# Matrix Product State classifier for MNIST

## Introduction
Tensor networks are sparse data structures that have been originally developed in the area of 
computational many-body quantum physics.

The aim of this notebook is to explain in a bit of detail how an MPS classifier works, and provide 
some details of how to train it. As an example we will use the MNIST data set of hand written digits,
and the fashion-MNIST dataset of fashion products. 

The content here follows closely the basic proposal by Miles Stoudenmire (arXiv: ...), but differs from
it in a few, but important aspect, and  I will explain these in more detail below. At this point, I'll just
summarize the advantages of my implementation as compared to Miles':
1. The network is inherently normalized, i.e. we don't need to regularize it
2. Overflow and underflow issues are eliminated. In particular, this includes:
    - normalization constants of projections of images into left and right environments
      can be rigorously ignored
    - simple gradient descent optimization is stable, i.e. it does not diverge if too large step sizes 
      are taken


## Background
This notebook implements a linear classifier for classifying handwritten digits using a supervised learning 
approach. The generic supervised learning setup consists of pairs $(x_n, y_n$) of data points.
$x_n$ here is a vector of features of the $n$-th input data (or sample), and $y_n$ is a
class label for this data point. In our example below, $x_n$ are the grey-scale values of a 2-d 
array of pixels of an image of a handwritten digit (with digit-values between 0 andn 9), 
and $y_n$ is the label indicating which digits is shown in the image. A supervised learning task 
then consists of finding a mapping $\tilde y_n \equiv f(x_n)$ which assigns one of $L$ class labels to any
input data $x_n$. The assigned label $\tilde y_n$ should as often as possible coincide with the 
correct label $y_n$. 

Our implementation below consists of two steps. The first step is a so called feature embedding step.
Feature embedding here means that prior to feeding our data $x_n$ into our classification function $f$, 
we use a non-linear embedding function $\tilde x_n \equiv \Phi(x_n)$ to map the datapoints $x_n$ into a high dimensional space. In general, there are many choices of functions $\phi(\cdot)$ one could use. We will below
use an embedding that has been proposed by Stoudenmire, but other choices are possible as well. 
If our input data consists of values $x_n = (x_n^1, x_n^2, \cdots, x_n^P)$
of $P$ greyscale values, then the embedding function $\Phi$ is given by the outer product of $P$ local
embedding functions $\phi_k(\cdot)$. The local embedding function $\phi_k(\cdot)$ maps the value of pixel $k$
into a two-dimensional vector space:
\begin{align}
\phi_k^{i_k}:\mathbb{R}\rightarrow \mathbb{C}^2\\
x^k \rightarrow \big(\cos(\frac{\pi}{2}x^k), \sin(\frac{\pi}{2}x^k)\big) =(\phi^1_k(x^k), \phi^2_k(x^k))\equiv
\vec \phi_k(x^k)
\end{align}
(not that $k$ here is a superscript and not an exponent). The map $\Phi(\cdot)$ is then  given by 
In the following we'll drop the subscript $k$ from $\phi$ to avoid clutter of notation.
\begin{align}
\Phi:\mathbb{R}^P\rightarrow \mathbb{C}^{2^P}\\
\tilde x_n \equiv \Phi(x_n) = \vec\phi(x^1_n)\otimes \cdots \otimes \vec\phi(x^P_n)=
\Phi^{i_1i_2\cdots i_P}(x_n) = \phi^{i_1}(x^1_n)\otimes \cdots \otimes \phi^{i_P}(x^P_n)
%(\cos(\frac{\pi}{2}x^1), sin(\frac{\pi}{2}x^1))\otimes 
%(\cos(\frac{\pi}{2}x^2), sin(\frac{\pi}{2}x^2))\otimes \cdots \otimes(\cos(\frac{\pi}{2}x^P), sin(\frac{\pi}{2}x^P))
\end{align}
where we have introduced two different ways of denoting $\Phi$.

In the second step we will use a linear function $f(\cdot)$ to map the embedded feature vector $\tilde x_n$ 
back into an $L$ dimensional vector space. 
In the implementation we're using below, we will then interpret a given orthogonal basis in this vector space
as corresponding to the class labels. We'll use below a so called one-hot encoding, which means that 
a vector $(1,0,\cdots,0)$ corresponds to label $1$, $(0,1,0\cdots 0)$ to label 2,  $(0,0,1,0\cdots 0)$ to label 3, 
and so on. Thus, the ideal mapping for any input data vector $x_n$ would be given by 
\begin{align}
x_n \rightarrow \tilde y_n = f(\Phi(x_n)) = \delta_{y_n,l}
\end{align}
where $\delta_{y_n,l}$ is the Kronecker delta function ($l$ labels the index in the final $L$ dimensional 
vector space). The goal is now to find a function $f$ which approximates the ideal mapping as good as possible.
In this notebook we will ilustrate how to use Matrix Product States (MPS) to implement this linear function $f$.



### Matrix Product States
Matrix product states (MPS) are efficient parametrizations of vectors in high-dimensional vector spaces.
Originally they have been developed to efficiently approximate ground-states of many-body quantum systems
in condensed matter physics, but they are now used well outside their original scope.
For the purpose of this notebook an MPS can be regarded as a high-dimensional parametrization of a tensor:
\begin{align}
T_{i_1 i_2 \cdots i_P} =  \sum_{\{\alpha_k\}} A^{i_1}_{\alpha_1} A^{i_2}_{\alpha_1\alpha_2}\dots A^{i_{P-1}}_{\alpha_{P-2}\alpha_{P-1}} A^{i_P}_{\alpha_{P-1}}
\end{align}
The $i_k$ are integers taking values in  $\{0,1\}$, and $A^{i_k}_{\alpha_{k-1}\alpha_k}$ are rank-3 tensors
with shapes $(D_{k-1},2,D_{k})$ for the index triple $(\alpha_{k-1}, i_k, \alpha_k)$
The tensors $A^{i_k}_{\alpha_{k-1}\alpha_k}$ will be optimized below.




### Classifier model
The clasifier in this notebook is given by an MPS of length $P+1$. The additional site
is the label tensor, which I'll denote with $B^{i_l}_{\alpha_{l-1}\alpha_l}$. 
Here $i_l$ runs from 1 to $L$ ($L=10$ for MNIST and fashion MNIST).
The label tensor can be shifted around in the code. 
The classifier is given by
\begin{align}
T_{i_1 i_2 \cdots i_P i_{P+1}} =  \sum_{\{\alpha_k\}} A^{i_1}_{\alpha_1} A^{i_2}_{\alpha_1\alpha_2}\dots A^{i_{P-1}}_{\alpha_{P-2}\alpha_{P-1}} A^{i_P}_{\alpha_{P-1}\alpha_P}B^{i_{P+1}}_{\alpha_P\alpha_{P+1}}
\end{align}
where we have arbitrarily put the label tensor to the right end of the MPS.
Classification is done by contracting the embedded image with the classifier and normalizing the result:

\begin{align}
f^{i_{P+1}}(\Phi(x_n)) = \sum_{i_k \neq i_{P+1}} T_{i_1 i_2 \cdots i_P i_{P+1}} \Phi^{i_1i_2\cdots i_P}(x_n)\\
\tilde y_n^{i_{P+1}} = \frac{f^{i_{P+1}}(\Phi(x_n)) }{\sqrt{\sum_{i_{P+1}}f^{i_{P+1}}(\Phi(x_n)) f^{i_{P+1}}(\Phi(x_n)) }}
\end{align}
As cost function we use the mean squared error:

\begin{align}
K = \frac{1}{2N}\sum_{n, i_{P+1}} (\tilde y_n^{i_{P+1}} - \delta_{y_ni_{P+1}})^2
\end{align}
The gradients can be worked out relatively easily (I'll add that later). One of the advantages of
the normalization is that 1) left and right embedded images (see the original publication) can be 
normalized, and the normalizatio constant can be discarded (it will drop out of all equations)
and 2) we don't need to explicitly regularize the network, which has beneficial effects on the optimization. 

dependencies:
you need `tensorflow 1.13` or higher, `numpy`, `TensorNetwork` and `matplotlib`.

In [ ]:
import tensorflow as tf
tf.enable_v2_behavior()
import tensornetwork as tn
import experiments.MPS_classifier.MPSMNIST as mm
import experiments.MPS.misc_mps as misc_mps
import experiments.MPS.matrixproductstates as MPS
import matplotlib.pyplot as plt
import numpy as np

misc_mps.compile_ncon(False)
%load_ext autoreload
%autoreload 2

In [ ]:
#mnist = tf.keras.datasets.mnist
mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')


Ntrain, Nx, Ny = x_train.shape

x_train = np.reshape(x_train,(Ntrain, Nx*Ny))
Ntest, Nx, Ny = x_test.shape
x_test = np.reshape(x_test,(Ntest, Nx*Ny))
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
#optional shuffling of the data
x_train, new_order = mm.shuffle(x_train)
x_test, new_order = mm.shuffle(x_test, new_order)

In [ ]:
plt.imshow(np.reshape(x_train[np.random.randint(0,x_train.shape[0]),:],(28,28)))

In [ ]:
#optional downsampling by averaging over a square of 4 pixels
def downsample(X):
    """
    optional downsampling step for the data in `X`. downsamples by averaging over a square of 4 pixels
    """
    Nt = X.shape[0]
    bla = np.pad(np.reshape(X,(Nt,28,28)),pad_width=((0,0),(1,1),(1,1)),mode='constant')
    rolled1 = np.roll(bla,shift=(0,-1,-1),axis=(0,1,2))
    rolled2 = np.roll(bla,shift=(0,0,-1),axis=(0,1,2))
    rolled3 = np.roll(bla,shift=(0,-1,0),axis=(0,1,2))
    av = (bla + rolled1 + rolled2 + rolled3)/4
    downsampled = av[:,1:-1:2, 1:-1:2]
    dim = downsampled.shape[1]
    return np.reshape(downsampled,(Nt,dim**2))
x_train= downsample(x_train)
x_test= downsample(x_test)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

prepare the samples into batches

In [ ]:
n_batches=50
train_data, train_labels = mm.generate_mapped_MNIST_batches(x_train,y_train,n_batches=n_batches,which='one_hot',scaling=1.0,  
                                              shuffle_pixels=False)
test_data, test_labels = mm.generate_mapped_MNIST_batches(x_test, y_test,n_batches=1)
N = data[0].shape[2]
print(N)

In [ ]:
D=20
mps = mm.MPSClassifier.eye(ds = [2]*N, D=D, num_labels=10,label_position = 0,
                                 scaling=1.0, name='test',dtype=dtype)
mps.position(len(mps)) #the mps has an orthogonality center that can be shifted around
mps.position(0)
mps.normalize()#normalize the center matrix

In [ ]:
batch = 0 #use batch zero for fast training
samples = tf.convert_to_tensor(train_data[batch].astype(dtype.as_numpy_dtype))
labels = tf.convert_to_tensor(train_labels[batch].astype(dtype.as_numpy_dtype))
test_d = tf.convert_to_tensor(test_data[0].astype(dtype.as_numpy_dtype))
test_l = tf.convert_to_tensor(test_labels[0].astype(dtype.as_numpy_dtype))
#mps.compute_data_environments(samples)

In [ ]:
plt.imshow(np.reshape(samples[np.random.randint(0,samples.shape[0]),1,:],(28,28)))

In [ ]:
mps.label_position(0,D=D)
mps.position(0)
mps.compute_data_environments(samples)

In [ ]:
l, a = mps.left_right_sweep_label(samples,labels,learning_rate=1E-2,D=D) #start with label tensor at the left and;
                                                                         #shift it to the right end during first sweep
#l, a = mps.right_left_sweep_label(samples,labels,learning_rate=1E-8,D=D)

In [ ]:
mps.label_position(392,D=D) #for fashion MNIST, leave the label tensor at the right end
mps.position(0)
mps.compute_data_environments(samples)

In [ ]:
n0 = 250 #for fashion MNIST it turns out to give better results to stop a bit away from the left end
for _ in range(10):
    l, a = mps.left_right_sweep_simple(samples,labels,learning_rate=1E-4)
    l, a = mps.right_left_sweep_simple(samples,labels,learning_rate=1E-4, n0=n0)